In [1]:
import pandas as pd
from sklearn.decomposition import NMF
import numpy as np
import random

In [2]:
movies = pd.read_csv("data_100k/movies.csv", index_col='movieId')
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('data_100k/ratings.csv', index_col = 'userId')
ratings.head()

,movieId,rating,timestamp
userId,,,
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931


In [4]:
tags = pd.read_csv('data_100k/tags.csv', index_col = 'userId')
tags.head()

,movieId,tag,timestamp
userId,,,
2,60756,funny,1445714994
2,60756,Highly quotable,1445714996
2,60756,will ferrell,1445714992
2,89774,Boxing story,1445715207
2,89774,MMA,1445715200


In [42]:
movies.shape

(9742, 2)

In [6]:
ratings.shape

(100836, 3)

In [7]:
tags.shape

(3683, 3)

In [8]:
joinedDF = pd.merge(movies,ratings, left_index = True,right_on ='movieId')
joinedDF.head()

,title,genres,movieId,rating,timestamp
userId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,847434962
7,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.5,1106635946
15,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,2.5,1510577970
17,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.5,1305696483


In [9]:
df = joinedDF[['title', 'rating']]
df.reset_index(inplace = True)
df.head()

,userId,title,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [10]:
df[df[['userId', 'title']].duplicated(keep = False)].sort_values(by = 'userId')

,userId,title,rating
80171,28,War of the Worlds (2005),3.5
88672,28,War of the Worlds (2005),3.5
80172,68,War of the Worlds (2005),2.0
88673,68,War of the Worlds (2005),2.5
69625,111,Confessions of a Dangerous Mind (2002),4.0
99483,111,Confessions of a Dangerous Mind (2002),4.0
19692,509,Emma (1996),3.5
78302,509,Emma (1996),3.5


In [11]:
duplicate_users = df[df[['userId', 'title']].duplicated(keep = 'first')].userId

In [12]:
test = df.drop_duplicates(['userId', 'title'])

In [13]:
test.head()

,userId,title,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [14]:
new_df = df[~df.userId.isin(duplicate_users)]
new_df.head()

,userId,title,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [15]:
new_df.shape

(97893, 3)

In [16]:
test.set_index(['userId', 'title']).rating.unstack()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [17]:
new_df.set_index(['userId', 'title']).rating.unstack()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [18]:
R_true = new_df.pivot(index = 'userId', columns = 'title', values = 'rating')
R_true.fillna(0, inplace = True)
R_true.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
R_true.columns

Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)', ''Tis the Season for Love (2015)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '(500) Days of Summer (2009)', '*batteries not included (1987)',
       ...
       'Zulu (2013)', '[REC] (2007)', '[REC]² (2009)',
       '[REC]³ 3 Génesis (2012)',
       'anohana: The Flower We Saw That Day - The Movie (2013)',
       'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)',
       '¡Three Amigos! (1986)', 'À nous la liberté (Freedom for Us) (1931)'],
      dtype='object', name='title', length=9582)

In [60]:
prof = pd.DataFrame(np.zeros(R_true.shape[1]))
prof = prof.T
#prof.columns = R_true.columns
prof.columns = R_true.columns
prof['Zulu (2013)']

0    0.0
Name: Zulu (2013), dtype: float64

In [61]:
user_ratings = {'Zulu (2013)':5}

In [81]:
for x,y in user_ratings.items():
    print(x,y)

Zulu (2013) 5


In [82]:
user = pd.DataFrame(np.zeros(R_true.shape[1]))
user = user.T
#prof.columns = R_true.columns
user.columns = R_true.columns

for film, rating in user_ratings.items():
    user[film] = rating

In [85]:
user['Zulu (2013)']

0    5
Name: Zulu (2013), dtype: int64

In [87]:
np.array(user)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [20]:
m = NMF(n_components=18, init='random', random_state=10)   # <-- the other dimension of P and Q; how many "genres"
m.fit(R_true)

NMF(alpha=0.0, beta_loss='frobenius', init='random', l1_ratio=0.0, max_iter=200,
    n_components=18, random_state=10, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [21]:
P = m.components_
P.shape

(18, 9582)

In [22]:
Q = m.transform(R_true)
Q.shape

(606, 18)

In [ ]:
np.dot(Q, P).round()

In [38]:
user = np.zeros(9582)   # user likes Tiffany but not Terminator
user[0]=0
user[1]=5
user[2]=5

In [34]:
user.shape

(9582,)

In [39]:
profile = m.transform(user.reshape(1,-1))
profile.shape

(1, 18)

In [40]:
movies_for_user = np.dot(P.T,profile.T) 

In [41]:
recommendation = pd.DataFrame(movies_for_user)
recommendation

,0
0,0.000000
1,0.000043
2,0.000036
3,0.000000
4,0.000000
...,...
9577,0.000027
9578,0.000000
9579,0.000000
9580,0.000101


In [ ]:
recommendation[recommendation[0] > 0]

In [ ]:
recommendation.sort_values(by = 0, ascending = False).head()

In [ ]:
recommendation.sort_values(by = 0, ascending = False).iloc[:5]

In [ ]:
R_true.columns[recommendation.sort_values(by = 0, ascending = False).head().index]

In [ ]:
R_true.columns[recommendation.sort_values(by = 0, ascending = False).head().index][0]

In [ ]:
print(m.reconstruction_err_)

In [ ]:
nR = np.dot(Q, P)
print(nR)

In [88]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [89]:
fuzz.partial_ratio("this is a test", "this is a test!")

100

In [94]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]
process.extract("new york jets", choices, limit=1)[0][0]


'New York Jets'

In [91]:
process.extractOne("cowboys", choices)

('Dallas Cowboys', 90)